### SageMaker Large Scale prediction

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


In [2]:
pytorch_custom_image_name="ppi-extractor:cpu-1.0.0-202012311208"
instance_type = "ml.m5.large" 

In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

### Step 1: Convert pubtator format to inference json

The input pubtator files look like this.. These are converted to produce inference 

```text
20791654|a|Liver scan characteristics and liver function tests of 72 patients with proved hepatic malignancy (54 metastatic, 18 primary) were evaluated. Well-defined focal defects were observed in 83% of patients with metastatic and 77% of patients with primary liver carcinoma. In 10% of the patients with metastatic liver disease the distribution of radioactivity was normal. Four or more biochemical liver function tests were normal in 33% of metastatic and 29% of primary liver cancer patients. Hepatic enlargement was present in the scan in 94% of the patients with liver metastases; however, data obtained from 104 necropsies of patients with hepatic metastases showed that only 46% had hepatomegaly. We recommend, therefore, that a liver scan should be performed before major tumour surgery in every patient with known malignancy regardless of normal liver size or normal liver function tests.
20791654	58	66	patients	Species	9606
20791654	193	201	patients	Species	9606
20791654	229	237	patients	Species	9606
20791654	282	290	patients	Species	9606
20791654	478	486	patients	Species	9606
20791654	546	554	patients	Species	9606
20791654	624	632	patients	Species	9606
20791654	796	803	patient	Species	9606

20791817|a|5-Aminosalicylic acid given to rats as a single intravenous injection led to necrosis of the proximal convoluted tubules and of the renal papilla. These two lesions developed at the same time and the cortical lesions did not appear to be a consequence of the renal papillary necrosis. Since the compound possesses the molecular structure both of a phenacetin derivative and of a salicylate these observations may be relevant to the problem of renal damage incident to abuse of analgesic compounds and suggest the possibility that in this syndrome cortical lesions may develop independently of renal papillary necrosis.
20791817	31	35	rats	Species	10116

```

In [4]:
import datetime
date_fmt = datetime.datetime.today().strftime("%Y%m%d%H")

In [11]:
#s3_input_pubtator = "s3://aegovan-data/pubmed_json_parts_annotation_iseries/pubmed19n0550.json.txt"
s3_input_pubtator = "s3://aegovan-data/pubmed_json_parts_annotation_iseries/"
s3_id_mapping_file="s3://aegovan-data/settings/HUMAN_9606_idmapping.dat"

s3_output_pubmed_asbtract = f"s3://aegovan-data/pubmed_asbtract/inference_multi_{date_fmt}/"

In [ ]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python3"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=10,
                                       max_runtime_in_seconds=172800,
                                       volume_size_in_gb = 50,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="ppi-large-inference-data-prep"


                                       )


sm_local_input_pubtator_txt = "/opt/ml/processing/input/data/json"
sm_local_input_idmapping = "/opt/ml/processing/input/data/mapping"
sm_local_output = "/opt/ml/processing/output"


script_processor.run(
        code='source/datatransformer/pubtator_annotations_inference_transformer.py',

        arguments=[
        
            sm_local_input_pubtator_txt,
            sm_local_output,
           "{}/{}".format(sm_local_input_idmapping,s3_id_mapping_file.split("/")[-1]) 

        ],
    
       inputs=[
                ProcessingInput(
                    source=s3_input_pubtator,
                    destination=sm_local_input_pubtator_txt,
                    s3_data_distribution_type="ShardedByS3Key")

            ,ProcessingInput(
                    source=s3_id_mapping_file,
                    destination=sm_local_input_idmapping,
                    s3_data_distribution_type="FullyReplicated")
            ],

        outputs=[ProcessingOutput(
                source=sm_local_output, 
                destination=s3_output_pubmed_asbtract,
                output_name='inferenceabstracts')]
    )

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  ppi-large-inference-data-prep-2020-12-31-12-29-53-262
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://aegovan-data/pubmed_json_parts_annotation_iseries/', 'LocalPath': '/opt/ml/processing/input/data/json', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'S3Input': {'S3Uri': 's3://aegovan-data/settings/HUMAN_9606_idmapping.dat', 'LocalPath': '/opt/ml/processing/input/data/mapping', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-324346001917/ppi-large-inference-data-prep-2020-12-31-12-29-53-262/input/code/pubtator_annotations_inference_transformer.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}

## Step 2: Run predictions

In [ ]:
prepare_models=False

In [7]:
jobs = [
"ppi-bert-2020-12-28-06-14-27-510",
"ppi-bert-2020-12-28-06-13-28-613",
"ppi-bert-2020-12-28-06-12-43-937",
"ppi-bert-2020-12-28-06-11-48-471",
"ppi-bert-2020-12-28-06-10-53-005",
"ppi-bert-2020-12-28-06-10-00-183",
"ppi-bert-2020-12-28-06-09-00-491",
"ppi-bert-2020-12-28-06-08-02-139",
"ppi-bert-2020-12-28-06-07-01-234",
"ppi-bert-2020-12-28-06-06-07-198"
]

s3_model_path_format = "s3://aegovan-data/results/{}/output/model.tar.gz"

s3_model_paths = [s3_model_path_format.format(j) for j in jobs]

In [8]:
s3_output_ensemble_models= "s3://aegovan-data/ensemble_models/{}".format("2020-12-28-06-part")

### Prepare ensemble models
TODO: This is just a hack to untar a bunch of zipped models and upload them to a single s3 locaton. Have a single processing job to this this is an overkill...

In [ ]:
def get_processing_inputs_s3_local_path(s3_model_paths, sm_local_input):
    # Map the s3 model path to local input path
    inputs = []
    for i, s3_path in enumerate(s3_model_paths):
         p = ProcessingInput(
                        source=s3_path,
                        destination="{}/{}".format(sm_local_input.rstrip("/"), i)
         )
         inputs.append(p)
    return inputs


In [ ]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor


sm_local_input = "/opt/ml/processing/input/models"
sm_local_output = "/opt/ml/processing/output"

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python3"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=1,
                                       max_runtime_in_seconds=172800,
                                       volume_size_in_gb = 50,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="ppi-ensemble-model-packer"
                                       )

if prepare_models:
    # Work around to get over the processing job input limit size
    chunk_size=5
    for i in range(0, len(s3_model_paths), chunk_size ):

        script_processor.run(
                code='source/algorithms/ensemble_inference_prepare_models.py',

                arguments=[
                    "--input-dir",
                    sm_local_input,
                    "--dest-dir",
                    sm_local_output

                ],

                inputs=get_processing_inputs_s3_local_path(s3_model_paths[i:i+chunk_size], sm_local_input),


                outputs=[ProcessingOutput(
                        source=sm_local_output, 
                        destination=s3_output_ensemble_models,
                        output_name='models')]
            )


### Run ensemble prediction

In [ ]:
s3_output_predictions = "s3://aegovan-data/pubmed_asbtract/predictions_multi_{}/".format("2020-12-28-06-part")

In [ ]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-202012311208"
instance_type = "ml.p3.8xlarge" 

In [ ]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python3"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=10,
                                       max_runtime_in_seconds=172800,
                                       volume_size_in_gb = 50,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="ppi-ensemble-inference"
                                       )


sm_local_input_models = "/opt/ml/processing/input/models"
sm_local_input_data = "/opt/ml/processing/input/models"
sm_local_output = "/opt/ml/processing/output"

main_predict.py PpiMulticlassDatasetFactory  ${datajson}  ${artefactsdir}   ${outdir}  

script_processor.run(
        code='source/main_predict.py',

        arguments=[
            "PpiMulticlassDatasetFactory",
            sm_local_input_data,
            sm_local_input_models,
            sm_local_output
        ],

        inputs=[
                ProcessingInput(
                    source=s3_output_pubmed_asbtract,
                    destination=sm_local_input_data,
                    s3_data_distribution_type="ShardedByS3Key")

            ,ProcessingInput(
                    source=s3_output_ensemble_models,
                    destination=sm_local_input_models,
                    s3_data_distribution_type="FullyReplicated")
            ],


        outputs=[ProcessingOutput(
                source=sm_local_output, 
                destination=s3_output_predictions,
                output_name='predictions')]
    )


